In [1]:
print("1")


1


In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad


In [3]:
# Allocate the paths to the directories
directory1 = "/scratch/bvdberg/SoloTE/run_A018/A018_SoloTE_output/A018_locustes_MATRIX/"
directory2 = "/scratch/bvdberg/SoloTE/run_A019/A019_SoloTE_output/A019_locustes_MATRIX/"
directory3 = "/scratch/bvdberg/SoloTE/run_A020/A020_SoloTE_output/A020_locustes_MATRIX/"


In [4]:
# Read in each matrix with their accompanying barcodes and features
adata1 = sc.read_mtx(directory1+"matrix.mtx")
print(adata1.obs_names[:10])
print(adata1.var_names[:10])
barcode1 = pd.read_csv(directory1+"barcodes.tsv", header=None, names=["barcode"])
feature1 = pd.read_csv(directory1+"features.tsv", sep="\t", header=None, names=["gene_name"])

# To resolve a ValueError: Length of passed value for obs_names is 114950, but this AnnData has shape: (159348, 114950) 
# We transpose the data so the axis are swapped with each other so they correspond with the lenghts of the features and barcodes
adata1 = adata1.transpose()

# Allocate the features and barcodes to the respective annotation data
adata1.obs_names = barcode1["barcode"]
adata1.var_names = feature1["gene_name"]

adata2 = sc.read_mtx(directory2+"matrix.mtx")
barcode2 = pd.read_csv(directory2+"barcodes.tsv", header=None, names=["barcode"])
feature2 = pd.read_csv(directory2+"features.tsv", sep="\t", header=None, names=["gene_name"])

adata2 = adata2.transpose()

adata2.obs_names = barcode2["barcode"]
adata2.var_names = feature2["gene_name"]

adata3 = sc.read_mtx(directory3+"matrix.mtx")
barcode3 = pd.read_csv(directory3+"barcodes.tsv", header=None, names=["barcode"])
feature3 = pd.read_csv(directory3+"features.tsv", sep="\t", header=None, names=["gene_name"])

adata3 = adata3.transpose()

adata3.obs_names = barcode3["barcode"]
adata3.var_names = feature3["gene_name"]


Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='object')
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='object')


In [5]:
print("adata1")
print(adata1.obs_names[10:])
print(adata1.var_names[10:])
# print("adata2")
# print(adata2.obs_names[:10])
# print(adata2.var_names[:10])
# print("adata3")
# print(adata3.obs_names[:10])
# print(adata3.var_names[:10])


adata1
Index(['ACACCGGTCCAGAGGA-1', 'ACAGCCGGTAAGTGTA-1', 'ACAGCTAGTAGATTAG-1',
       'ACATCAGGTTGCCTCT-1', 'ACATGGTTCTCCTATA-1', 'ACGAGGAAGTGTTAGA-1',
       'ACGAGGAGTACAGACG-1', 'ACGCCAGCAGAGTGTG-1', 'ACGGGTCTCTAGAGTC-1',
       'ACGTCAAGTATAGTAG-1',
       ...
       'CTGGTCTTCCTATGTT-1', 'GGCCGATAGTATTGGA-1', 'CTTACCGCACGTCTCT-1',
       'AGCTTGACACCATCCT-1', 'ATTCTACTCCCTCTTT-1', 'CCTTCCCTCATGTCTT-1',
       'GCTGCTTGTCTCTCTG-1', 'GTTTCTATCTCTGTCG-1', 'TTCGGTCTCTAACTCT-1',
       'TTGCCGTCACTCTGTC-1'],
      dtype='object', name='barcode', length=118278)
Index(['0610039K10Rik', '0610040B10Rik', '0610040F04Rik', '0610040J01Rik',
       '0610043K17Rik', '1010001B22Rik', '1100001I12Rik', '1110002J07Rik',
       '1110002L01Rik', '1110004F10Rik',
       ...
       'SoloTE|chrY|90804592|90804705|B1_Mus2:Alu:SINE|19.1|-',
       'SoloTE|chrY|90804713|90805946|L1_Mm:L1:LINE|25.1|-',
       'SoloTE|chrY|90806301|90806444|B1_Mus2:Alu:SINE|18.8|-',
       'SoloTE|chrY|90806863|90806976|B1_

In [6]:
# Give each dataset their own label to keep track of where they came from
adatas = {"A018": adata1, "A019": adata2, "A020": adata3}

# Combine the adata sets to one data set with the Concatenate function, we use 'outer' to preserve as much data as possible. Missing variablles will become NaN values
adatas = ad.concat(adatas, label="dataset_origin",  join="outer")
# adatas.obs_names_make_unique()


/mbshome/bvdberg/miniconda3/envs/scanpy/lib/python3.11/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [7]:
print (adata1)
print (adata2)
print (adata3)
print (adatas)


AnnData object with n_obs × n_vars = 118288 × 156966
AnnData object with n_obs × n_vars = 114950 × 159348
AnnData object with n_obs × n_vars = 123423 × 164554
AnnData object with n_obs × n_vars = 356661 × 313925
    obs: 'dataset_origin'


In [8]:
bonemarrow_h5ad = ad.read_h5ad('/scratch/bvdberg/Data/bonemarrow_m.h5ad')


In [9]:
print (bonemarrow_h5ad)


AnnData object with n_obs × n_vars = 14434 × 2000
    obs: 'nCount_RNA', 'nFeature_RNA', 'sampleID', 'tissue', 'model', 'age', 'model_age', 'mouseID', 'doublet_scores', 'percent.mt', 'percent.ribo', 'percent.hb', 'percent.hist', 'S.Score', 'G2M.Score', 'group_umap_sub', 'annotation_lv0', 'annotation_lv1', 'annotation_lv2', 'annotation_lv3', 'phase'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'
    obsm: 'X_tsne', 'X_umap', 'X_umap_sub', 'X_umap_tissue'


In [ ]:
old_spf_bonemarrow_adata = ad.concat([bonemarrow_h5ad,adatas],join="outer")
